## Imports

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=1
%matplotlib inline

In [ ]:
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
import tensorflow as tf
from data.dataset import Dataset
import data.data_selector as ds
import utils.data_processing as dp
import utils.plot_functions as pf
import analysis.analysis_picker as ap

In [ ]:
class mlp_params(object):
  def __init__(self):
    self.model_type = "mlp"
    self.model_name = "mlp_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

class ae_deep_params(object):
  def __init__(self):
    self.model_type = "ae"
    self.model_name = "ae_deep_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

params_list = [ae_deep_params()]
for params in params_list:
  params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)
  
analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]

In [ ]:
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.setup(params)
  analyzer.setup_model(analyzer.model_params)
  analyzer.load_analysis(save_info=params.save_info)
  analyzer.model_name = params.model_name

In [ ]:
for analyzer in analyzer_list:
  pre_images = np.stack([analyzer.neuron_vis_output["optimal_stims"][target_id][-1].reshape(28,28)
    for target_id in range(len(analyzer.analysis_params.neuron_vis_targets))], axis=0)
  pre_image_fig = pf.plot_weights(pre_images, title=analyzer.model_name+" pre-images", figsize=(4,8))
  pre_image_fig.savefig(analyzer.analysis_out_dir+"/vis/pre_images.png",
      transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
target_neuron_idx = 0
step_idx = -1

for analyzer in analyzer_list:
  analyzer.bf_id0 = target_neuron_idx
  analyzer.bf0 = analyzer.neuron_vis_output["optimal_stims"][analyzer.bf_id0][step_idx]
  analyzer.bf0 = analyzer.bf0.reshape(np.prod(analyzer.model.get_input_shape()[1:]))
  analyzer.bf0 = analyzer.bf0 / np.linalg.norm(analyzer.bf0)
  
  fig, axes = plt.subplots(1, 2, figsize=(10,4))
  
  ax = pf.clear_axis(axes[0])
  ax.imshow(analyzer.bf0.reshape(28, 28), cmap="Greys_r")#, vmin=0.0, vmax=1.0)
  ax.set_title("Optimal\ninput image")
  
  axes[1].plot(analyzer.neuron_vis_output["loss"][analyzer.bf_id0])
  axes[1].set_title("Optimization loss")
  
  plt.show()

In [ ]:
def find_orth(matrix):
  rand_vect = np.random.rand(matrix.shape[0], 1)
  new_matrix = np.hstack((matrix, rand_vect))
  candidate_vect = np.zeros(matrix.shape[1]+1)
  candidate_vect[-1] = 1
  orth_vect = np.linalg.lstsq(new_matrix.T, candidate_vect, rcond=None)[0]
  orth_vect = np.squeeze((orth_vect  / np.linalg.norm(orth_vect)).T) 
  return orth_vect

In [ ]:
for analyzer in analyzer_list:
  analyzer.orth_col_matrix = analyzer.bf0.T[:,None]
  analyzer.rand_num_orthogonal = np.prod(analyzer.model.get_input_shape()[1:])-1

  for pop_idx in range(analyzer.rand_num_orthogonal):
    v = find_orth(analyzer.orth_col_matrix)
    analyzer.orth_col_matrix = np.append(analyzer.orth_col_matrix, v[:,None], axis=1)

  if all(np.abs(np.dot(analyzer.bf0, col)) < 1e-9 for col in analyzer.orth_col_matrix[:,1:].T):
    print("Success")
  else:
    count = np.sum([int(np.abs(np.dot(analyzer.bf0, col)) < 1e-9) for col in analyzer.orth_col_matrix[:,1:].T])
    print("Failure,", count, "were non-orthogonal")

In [ ]:
def set_norm_activity(analyzer, target_neuron_idx, num_rand_orthogonal, orthogonal_idx, num_imgs):
  # Get target neuron pre-image
  analyzer.bf_id0 = target_neuron_idx
  analyzer.bf0 = analyzer.neuron_vis_output["optimal_stims"][analyzer.bf_id0][step_idx]
  analyzer.bf0 = analyzer.bf0.reshape(np.prod(analyzer.model.get_input_shape()[1:]))
  analyzer.bf0 = analyzer.bf0 / np.linalg.norm(analyzer.bf0)

  # Get orthogonal vectors
  orth_col_matrix = analyzer.bf0.T[:,None]
  for pop_idx in range(num_rand_orthogonal):
    v = find_orth(orth_col_matrix)
    orth_col_matrix = np.append(orth_col_matrix, v[:,None], axis=1)

  # Construct image dataset
  x_pts = np.linspace(-2.0, 2.0, int(np.sqrt(num_imgs)))
  y_pts = np.linspace(-2.0, 2.0, int(np.sqrt(num_imgs)))
  X_mesh, Y_mesh = np.meshgrid(x_pts, y_pts)
  analyzer.proj_datapoints = np.stack([X_mesh.reshape(num_imgs), Y_mesh.reshape(num_imgs)], axis=1)
  
  proj_matrix, v = analyzer.bf_projections(analyzer.bf0, np.squeeze(orth_col_matrix[:, orthogonal_idx]))
  analyzer.proj_neuron0 = np.dot(proj_matrix, analyzer.bf0).T
  analyzer.proj_neuron1 = np.dot(proj_matrix, np.squeeze(orth_col_matrix[:, orthogonal_idx])).T
  
  datapoints = np.stack([np.dot(proj_matrix.T, analyzer.proj_datapoints[data_id,:])
    for data_id in range(num_imgs)]) #inject
  datapoints, orig_shape = dp.reshape_data(datapoints, flatten=False)[:2]
  datapoints = {"test": Dataset(datapoints, lbls=None,
    ignore_lbls=None, rand_state=analyzer.rand_state)}
  params={"whiten_data":analyzer.model_params.whiten_data}
  if params["whiten_data"]:
    params["whiten_method"] = analyzer.model_params.whiten_method
  datapoints = analyzer.model.preprocess_dataset(datapoints, params=params)
  datapoints = analyzer.model.reshape_dataset(datapoints, analyzer.model_params)
  datapoints["test"].images /= np.max(np.abs(datapoints["test"].images))
  datapoints["test"].images *= analyzer.analysis_params.input_scale
  
  activations = analyzer.compute_activations(datapoints["test"].images)#, batch_size=num_imgs//16)
  activity_max = np.amax(np.abs(activations))
  analyzer.norm_activity = activations / (activity_max + 0.00001) # Rescale between -1 and 1

In [ ]:
num_rand_orthogonal = np.prod(analyzer.model.get_input_shape()[1:])
set_norm_activity(
  analyzer=analyzer_list[0],
  target_neuron_idx=0,#np.random.choice(range(analyzer.model.get_num_latent()), 1),
  num_rand_orthogonal=num_rand_orthogonal,
  orthogonal_idx=np.random.choice(range(1, num_rand_orthogonal), 1),
  num_imgs=int(228**2))

In [ ]:
for analyzer in analyzer_list:
  num_plots_y = 1
  num_plots_x = 2
  gs1 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=0.3, width_ratios=[4, 1])
  fig = plt.figure(figsize=(6,6))
  curve_ax = pf.clear_axis(fig.add_subplot(gs1[0]))
  #cmap = plt.get_cmap('tab20b')
  cmap = plt.get_cmap('viridis')
  vmin = np.floor(np.min(analyzer.norm_activity))#0.0
  vmax = np.ceil(np.max(analyzer.norm_activity))#1.0
  
  #name_suffix = "continuous"
  #pts = curve_ax.scatter(analyzer.proj_datapoints[:,0], analyzer.proj_datapoints[:,1],
  #  vmin=vmin, vmax=vmax, cmap=cmap, alpha=0.5, c=analyzer.norm_activity[:, analyzer.bf_id0], s=5.0)
  
  norm_activity = analyzer.norm_activity[:, analyzer.bf_id0]
  norm_activity = norm_activity.reshape(int(np.sqrt(num_imgs)), int(np.sqrt(num_imgs)))
  
  levels = 5
  name_suffix = ""
  contsf = curve_ax.contourf(X_mesh, Y_mesh, norm_activity,
    levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)
  
  curve_ax.arrow(0, 0, analyzer.proj_neuron0[0].item(), analyzer.proj_neuron0[1].item(),
    width=0.05, head_width=0.15, head_length=0.15, fc='r', ec='r')
  curve_ax.arrow(0, 0, analyzer.proj_neuron1[0].item(), analyzer.proj_neuron1[1].item(),
    width=0.05, head_width=0.15, head_length=0.15, fc='w', ec='k')
  
  curve_ax.set_ylim([-2, 2.0])
  curve_ax.set_xlim([-2, 2.0])
  curve_ax.set_aspect("equal")
  curve_ax.set_title("Neuron ID "+str(analyzer.bf_id0))
  
  gs2 = gridspec.GridSpecFromSubplotSpec(2, 1, gs1[1], hspace=-0.5)
  bf1_ax = pf.clear_axis(fig.add_subplot(gs2[0]))
  bf1_ax.imshow(analyzer.neuron_vis_output["optimal_stims"][analyzer.bf_id0][step_idx].reshape((28,28)),
    cmap="Greys_r")
  bf1_ax.set_title("Primary\n Stimulus", color='r', fontsize=16)
  
  activity_ax = fig.add_subplot(gs2[1])
  activity_ax.plot(norm_activity[0,:], color='k')
  activity_ax.set_aspect(1.0/activity_ax.get_data_ratio())
  activity_ax.set_title("Activity along\nstimulus vector")
  
  fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_response_contours_bf0id"+str(analyzer.bf_id0)+name_suffix+".png",
    transparent=True, bbox_inches="tight", pad_inches=0.01)
  plt.show()

In [ ]:
num_neurons = 5
num_orth_directions = 8


num_plots_y = 1
num_plots_x = 2
gs1 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=0.3, width_ratios=[4, 1])
fig = plt.figure(figsize=(6,6))
curve_ax = pf.clear_axis(fig.add_subplot(gs1[0]))
#cmap = plt.get_cmap('tab20b')
cmap = plt.get_cmap('viridis')
vmin = np.floor(np.min(analyzer.norm_activity))#0.0
vmax = np.ceil(np.max(analyzer.norm_activity))#1.0

#name_suffix = "continuous"
#pts = curve_ax.scatter(analyzer.proj_datapoints[:,0], analyzer.proj_datapoints[:,1],
#  vmin=vmin, vmax=vmax, cmap=cmap, alpha=0.5, c=analyzer.norm_activity[:, analyzer.bf_id0], s=5.0)

norm_activity = analyzer.norm_activity[:, analyzer.bf_id0]
norm_activity = norm_activity.reshape(int(np.sqrt(num_imgs)), int(np.sqrt(num_imgs)))

levels = 5
name_suffix = ""
contsf = curve_ax.contourf(X_mesh, Y_mesh, norm_activity,
  levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)

curve_ax.arrow(0, 0, analyzer.proj_neuron0[0].item(), analyzer.proj_neuron0[1].item(),
  width=0.05, head_width=0.15, head_length=0.15, fc='r', ec='r')
curve_ax.arrow(0, 0, analyzer.proj_neuron1[0].item(), analyzer.proj_neuron1[1].item(),
  width=0.05, head_width=0.15, head_length=0.15, fc='w', ec='k')

curve_ax.set_ylim([-2, 2.0])
curve_ax.set_xlim([-2, 2.0])
curve_ax.set_aspect("equal")
curve_ax.set_title("Neuron ID "+str(analyzer.bf_id0))

In [ ]:
"""
* Compute a unit vector that is in the same plane as a given basis function pair (B1,B2) and is orthogonal to B1, where B1 is the target basis for comparison and B2 is selected from all other bases.
* Construct a line of data points in this plane
* Project the data points into image space, compute activations, plot activations
"""
for analyzer in analyzer_list:
  analyzer.pop_num_imgs = 100
  
  orthogonal_list = [idx for idx in range(analyzer.bf_stats["num_outputs"])]
  num_orthogonal = len(orthogonal_list)
  
  pop_x_pts = np.linspace(-2.0, 2.0, int(analyzer.pop_num_imgs))
  pop_y_pts = np.linspace(-2.0, 2.0, int(analyzer.pop_num_imgs))
  pop_X, pop_Y = np.meshgrid(pop_x_pts, pop_y_pts)
  pop_proj_datapoints = np.stack([pop_X.reshape(analyzer.pop_num_imgs**2), pop_Y.reshape(analyzer.pop_num_imgs**2)], axis=1) # construct a grid
  
  #x_target = pop_x_pts[int(6*analyzer.pop_num_imgs/8)] # find a location to take a slice
  x_target = pop_x_pts[int(0.25*analyzer.pop_num_imgs)] # find a location to take a slice
  
  slice_indices = np.where(pop_proj_datapoints[:,0]==x_target)[0]
  analyzer.pop_proj_datapoints = pop_proj_datapoints[slice_indices,:] # slice grid
  
  pop_datapoints = [None,]*num_orthogonal
  #pop_proj_neurons = [None,]*num_orthogonal
  for pop_idx, tmp_bf_id1 in enumerate(orthogonal_list):
    tmp_bf1 = analyzer.bf_stats["basis_functions"][tmp_bf_id1].reshape((analyzer.model_params.num_pixels))
    tmp_bf1 /= np.linalg.norm(tmp_bf1)
    tmp_proj_matrix, v = analyzer.bf_projections(analyzer.bf0, tmp_bf1) 
    pop_datapoints[pop_idx] = np.dot(pop_proj_datapoints, tmp_proj_matrix)#[slice_indices,:]
  
  pop_datapoints = np.reshape(np.stack(pop_datapoints, axis=0),
    [num_orthogonal*analyzer.pop_num_imgs, analyzer.model_params.num_pixels])
  
  pop_datapoints = dp.reshape_data(pop_datapoints, flatten=False)[0]
  analyzer.pop_datapoints = {"test": Dataset(pop_datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
  analyzer.pop_datapoints = analyzer.model.preprocess_dataset(analyzer.pop_datapoints,
    params={"whiten_data":analyzer.model_params.whiten_data,
    "whiten_method":analyzer.model_params.whiten_method})
  analyzer.pop_datapoints = analyzer.model.reshape_dataset(analyzer.pop_datapoints, analyzer.model_params)
  analyzer.pop_datapoints["test"].images /= np.max(np.abs(analyzer.pop_datapoints["test"].images))
  analyzer.pop_datapoints["test"].images *= analyzer.analysis_params.input_scale

In [ ]:
for analyzer in analyzer_list:
  pop_activations = analyzer.compute_activations(analyzer.pop_datapoints["test"].images)[:, analyzer.bf_id0]
  pop_activations = pop_activations.reshape([num_orthogonal, pop_num_imgs])
  analyzer.pop_norm_activity = pop_activations / np.amax(np.abs(pop_activations))

In [ ]:
"""
* Construct the set of unit-length bases that are orthogonal to B0 (there should be B0.size-1 of them)
* Construct a line of data points in each plane defined by B0 and a given orthogonal basis
* Project the data points into image space, compute activations, plot activations
"""
for analyzer in analyzer_list:
  analyzer.rand_pop_num_imgs = 100
  analyzer.rand_num_orthogonal = analyzer.bf_stats["num_inputs"]-1
  
  pop_x_pts = np.linspace(-2.0, 2.0, int(analyzer.rand_pop_num_imgs))
  pop_y_pts = np.linspace(-2.0, 2.0, int(analyzer.rand_pop_num_imgs))
  pop_X, pop_Y = np.meshgrid(pop_x_pts, pop_y_pts)
  analyzer.rand_pop_proj_datapoints = np.stack([analyzer.pop_X.reshape(analyzer.rand_pop_num_imgs**2),
    analyzer.pop_Y.reshape(analyzer.rand_pop_num_imgs**2)], axis=1) # construct a grid
  
  #x_target = pop_x_pts[int(6*analyzer.rand_pop_num_imgs/8)] # find a location to take a slice
  x_target = pop_x_pts[int(0.25*analyzer.rand_pop_num_imgs)] # find a location to take a slice
  
  slice_indices = np.where(analyzer.rand_pop_proj_datapoints[:,0]==x_target)[0]
  analyzer.rand_pop_proj_datapoints = analyzer.rand_pop_proj_datapoints[slice_indices,:] # slice grid
  
  analyzer.rand_pop_datapoints = [None,]*analyzer.rand_num_orthogonal
  for pop_idx in range(analyzer.rand_num_orthogonal):
    v = orth_col_matrix[:, pop_idx]
    tmp_proj_matrix = np.stack([analyzer.bf0, v], axis=0)
    analyzer.rand_pop_datapoints[pop_idx] = np.dot(analyzer.rand_pop_proj_datapoints,
      tmp_proj_matrix)#[slice_indices,:]

  analyzer.rand_pop_datapoints = np.reshape(np.stack(analyzer.rand_pop_datapoints, axis=0),
    [analyzer.rand_num_orthogonal*analyzer.rand_pop_num_imgs, analyzer.model_params.num_pixels])

  analyzer.rand_pop_datapoints = dp.reshape_data(analyzer.rand_pop_datapoints, flatten=False)[0]
  analyzer.rand_pop_datapoints = {"test": Dataset(analyzer.rand_pop_datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
  analyzer.rand_pop_datapoints = analyzer.model.preprocess_dataset(analyzer.rand_pop_datapoints,
    params={"whiten_data":False, "whiten_method":None})
  analyzer.rand_pop_datapoints = analyzer.model.reshape_dataset(analyzer.rand_pop_datapoints, analyzer.model_params)
  analyzer.rand_pop_datapoints["test"].images /= np.max(np.abs(analyzer.rand_pop_datapoints["test"].images))
  analyzer.rand_pop_datapoints["test"].images *= analyzer.analysis_params.input_scale

In [ ]:
analyzer.analysis_params.input_scale

In [ ]:
for analyzer in analyzer_list:
  rand_pop_activations = analyzer.compute_activations(analyzer.rand_pop_datapoints["test"].images)[:, analyzer.bf_id0]
  rand_pop_activations = rand_pop_activations.reshape([analyzer.rand_num_orthogonal, analyzer.rand_pop_num_imgs])
  analyzer.rand_pop_norm_activity = rand_pop_activations / np.amax(np.abs(rand_pop_activations))

In [ ]:
print(np.min(analyzer.norm_activity))
print(np.max(analyzer.norm_activity))
print(analyzer.norm_activity.shape)

In [ ]:
for analyzer in analyzer_list:
  analyzer.bf_coeffs = [
    np.polynomial.polynomial.polyfit(analyzer.pop_proj_datapoints[:,1], analyzer.pop_norm_activity[orthog_idx,:], deg=2)
    for orthog_idx in range(num_orthogonal)]
  analyzer.bf_fits = [
    np.polynomial.polynomial.polyval(analyzer.pop_proj_datapoints[:,1], coeff)
    for coeff in analyzer.bf_coeffs]
  analyzer.bf_curvatures = [np.polyder(fit, m=2) for fit in analyzer.bf_fits]
  
  analyzer.rand_coeffs = [np.polynomial.polynomial.polyfit(analyzer.rand_pop_proj_datapoints[:,1],
    analyzer.rand_pop_norm_activity[orthog_idx,:], deg=2) for orthog_idx in range(analyzer.rand_num_orthogonal)]
  analyzer.rand_fits = [np.polynomial.polynomial.polyval(analyzer.rand_pop_proj_datapoints[:,1], coeff)
    for coeff in analyzer.rand_coeffs]
  analyzer.rand_curvatures = [np.polyder(fit, m=2) for fit in analyzer.rand_fits]

In [ ]:
analyzer_idx = 0

bf_curvatures = np.stack(analyer_list[analyzer_idx].bf_coeffs, axis=0)[:,2]
rand_curvatures = np.stack(analyzer_list[analyzer_idx].rand_coeffs, axis=0)[:,2]

num_bins = 100
bins = np.linspace(-0.2, 0.2, num_bins)
bf_hist, bin_edges = np.histogram(bf_curvatures.flatten(), bins)
rand_hist, _ = np.histogram(rand_curvatures.flatten(), bins)
bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
bin_centers = bin_left + (bin_right - bin_left)/2

fig, ax = plt.subplots(1, figsize=(16,9))

ax.bar(bin_centers, rand_hist, width=0.0022, log=False, color="g", alpha=0.5, align="center", label="Random Projection")
ax.bar(bin_centers, bf_hist, width=0.0022, log=False, color="r", alpha=0.5, align="center", label="BF Projection")

ax.set_xticks(bin_left, minor=True)
ax.set_xticks(bin_left[::15], minor=False)
ax.xaxis.set_major_formatter(FormatStrFormatter("%0.3f"))
for tick in ax.xaxis.get_major_ticks():
  tick.label.set_fontsize(24) 
for tick in ax.yaxis.get_major_ticks():
  tick.label.set_fontsize(24) 

ax.set_title("Histogram of Curvatures", fontsize=32)
ax.set_xlabel("Curvature", fontsize=32)
ax.set_ylabel("Count", fontsize=32)
ax.legend(loc=2, fontsize=32)
fig.savefig(analyzer.analysis_out_dir+"/vis/histogram_of_curvatures_bf0id"+str(analyzer.bf_id0)+".png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)
plt.show()

In [ ]:
activations.shape